# Demo: create text grids 

Generate a regular lat/lon grid where **z = 1** (black) or **z = 0** (white), such that when plotted with a Robinson projection the user's text appears undistorted, for use in filtering tests

## Setup

In [ ]:
import sys, pathlib

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from dataclasses import asdict

from pytomofilt.textgrid import TextGridConfig, generate_text_grid, generate_multi_text_grid, save_grid_to_xyz_file, load_grid

## Define the plotting helper function

A simple comparison plot of a Cartesian (Plat Caree) vs Robinson projection,  to demonstrate that the text is undistorted in Robinson

In [ ]:
def plot_grid(df, central_meridian, title_extra=""):
    """Plot the grid in both Plate Carrée and Robinson projections."""
    # Reshape z back to a 2-D grid
    lons = np.sort(df["lon"].unique())
    lats = np.sort(df["lat"].unique())
    z = df.set_index(["lat", "lon"])["z"].unstack().values

    fig, axes = plt.subplots(
        1, 2, figsize=(18, 6),
        subplot_kw={"projection": None},  # overridden below
    )
    plt.subplots_adjust(wspace=0.05)

    projections = [
        ccrs.PlateCarree(),
        ccrs.Robinson(central_longitude=central_meridian),
    ]
    titles = [
        f"Plate Carrée {title_extra}",
        f"Robinson {title_extra}",
    ]

    for ax_placeholder, proj, title in zip(axes, projections, titles):
        # Replace the non-geo axis with a geo one
        pos = ax_placeholder.get_position()
        ax_placeholder.remove()
        ax = fig.add_axes(pos, projection=proj)

        ax.pcolormesh(
            lons, lats, z,
            transform=ccrs.PlateCarree(),
            cmap="Greys",
            shading="auto",
        )
        ax.coastlines(linewidth=0.5, color="steelblue")
        ax.set_global()
        ax.set_title(title, fontsize=12)

    plt.show()

# Step 1: layers with a single text object

Iterate over depth slices

In [ ]:
words_dict = {
    50: ["THE", 135],
    660: ["EARTH'S", 270],
    1000: ["LOWER", 270],
    1500: ["MANTLE", 270],
    2000: ["ROCKS", 270],
}

# Loop over the layers
for k, v in words_dict.items():

    config = TextGridConfig(
        text             = v[0],
        center_lon       = 135.0,       # centre longitude of text placement (degrees)
        center_lat       = 0.0,         # centre latitude of text placement (degrees)
        size_deg         = v[1],        # width of text bounding box in degrees of longitude
        central_meridian = 135.0,       # Robinson projection central meridian (degrees)
        grid_spacing_deg = 0.5,         # output grid spacing (degrees)
        font_path        = None,        # None = auto-download DejaVu Sans Bold
        font_size        = 1000,        # font rasterisation quality (pixels)
        fill_value       = 1,           # Amount to add to the grid inside the text        
    )

    # generate a pandas dataframe of the x y z values
    df = generate_text_grid(config)

    # save the grid to disk in the output folder
    output_path = f"output/depth_{k}.xyz"
    save_grid_to_xyz_file(df, output_path)

    # make a visualisation
    plot_grid(df, config.central_meridian, title_extra=f"{k} km")

# Step 2: CMB layer with two text objects

Use the generate_multi_text_grid function to combine multiple TextGridConfig objects.

Behaviour is that fill value is additive - such that if two text objects with fill value = 1 overlap, the result at the overlapping location will be 2.

In [ ]:
heart1 = TextGridConfig(
    text             = "♥",
    center_lon       = 10,
    center_lat       = -15,
    size_deg         = 60,
    central_meridian = 135.0,
    grid_spacing_deg = 0.5,
    font_path        = None,
    font_size        = 1000,
    fill_value       = 1
)

heart2 = TextGridConfig(
    text             = "♥",
    center_lon       = -150,
    center_lat       = -15,
    size_deg         = 60,
    central_meridian = 135.0,
    grid_spacing_deg = 0.5,
    font_path        = None,
    font_size        = 1000,
)

df = generate_multi_text_grid([heart1, heart2])
output_path = f"output/depth_2850.xyz"
save_grid_to_xyz_file(df, output_path)
plot_grid(df, config.central_meridian, title_extra=f"2850 km")